# Import all needed module in python

In [1]:
import numpy as np
import pandas as pd
import string,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.utils as ku
import tensorflow 
tensorflow.random.set_seed(2)
from numpy.random import seed
seed(1)

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

#  import data 

In [2]:
path = 'data/'
all_headlines = []

for filename in os.listdir(path):
    if 'Articles' in filename:
        articels_df = pd.read_csv(path+filename)
        all_headlines.extend(list(articels_df.headline.values))
        break
all_headlines = [x for x in all_headlines if x !='Unknown']

In [3]:
len(all_headlines)

831

# some data clean

In [4]:
def clean_text(txt):
    txt = ''.join(x for x in txt if x not in string.punctuation).lower()
    txt = txt.encode('utf8').decode('ascii','ignore')
    return txt
corpus = [clean_text(x) for x in all_headlines]

In [5]:
corpus[:5]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted']

In [6]:
tokenizer = Tokenizer()
def genaret_sequences_of_text(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index)+1
    
    input_sequences = []
    
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequences = token_list[:i+1]
            input_sequences.append(n_gram_sequences)
    return input_sequences, total_words

In [7]:
input_sequences, total_words = genaret_sequences_of_text(corpus)

In [8]:
total_words

2422

In [9]:
def genaret_pad_sequences(input_sequences):
    max_sequences_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequences_len,padding='pre'))
    
    predictor, labels = input_sequences[:,:-1], input_sequences[:,-1]
    labels = ku.to_categorical(labels, num_classes=total_words)
    
    return predictor, labels, max_sequences_len

In [10]:
predictor, labels, max_sequences_len = genaret_pad_sequences(input_sequences)

In [11]:
max_sequences_len

19

# Create a model 

In [12]:
def create_model(max_sequences_len, total_words):
    input_len = max_sequences_len-1
    
    model = Sequential()
    
    model.add(Embedding(total_words, 64, input_length=input_len))
    
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    model.add(Dense(total_words,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequences_len, total_words )

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 64)            155008    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2422)              244622    
Total params: 465,630
Trainable params: 465,630
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(predictor, labels, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [15]:
def test_model(seed_text, next_word, model, max_sequences_len):
    for _ in range(next_word):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequences_len-1,padding='pre', )
        predict = model.predict_classes(token_list, verbose=0)
    
        output = ''
        for word, index in tokenizer.word_index.items():
            if index==predict:
                output = word
                break
        seed_text += " "+output
    
    return seed_text.title()

In [17]:
print(test_model('shohel', 5, model,max_sequences_len ))
print(test_model('shakib', 6, model,max_sequences_len ))
print(test_model('Husna', 100, model,max_sequences_len ))

Shohel Lament Trumps Silence On Health
Shakib Lament Trumps Silence On Health Plan
Husna Lament Trumps Silence On Health Plan But On Solar Not Littering Arctic Waters But On Your Future Ignites Hot Hot Hot Him Him Think At Door End Him Him Him Him Him Him Him Him Him Him Him Him Are Are Are Flying At Any Allowed Industry Attack On Charging Allowed Person Over Niger About Days Over Can About Him Him Him Him Him Him Him Him Him Like All Are Coal Win At Him Him About In An Menu About At An Cross Hairs Thats Hairs Tough Him Him Him Him Flying Him Him Him Him Him Flying Him


In [20]:
print(test_model('', 10, model,max_sequences_len ))

Riya Khan Lament Trumps Silence On Health Plan But On Solar Not
